# Laboratorio: Setup y uso básico de LLMs con LangChain + Prompt engineering avanzado

Este laboratorio está pensado para completarse en ~2 horas. Trabajaremos en Google Colab con recursos gratuitos, usando la Inference API de Hugging Face y un modelo instruct abierto.

- Contenidos:
  - Setup en Colab y configuración de Hugging Face Inference API
  - Uso básico de LLMs con LangChain (LCEL)
  - Parámetros de decodificación y control de estilo
  - Prompt engineering avanzado: zero-shot, few-shot, Chain of Thought y salida estructurada (JSON)

Al finalizar, podrás:
- Conectarte a un LLM instruct vía Hugging Face Inference API desde LangChain.
- Construir cadenas simples con `prompt | llm | parser`.
- Diseñar prompts efectivos y controlar formato de salida (incluido JSON).


## Parte 0 — Setup (Colab + librerías + token)

Usaremos versiones estables para minimizar fricción en Colab. Asegurate de ejecutar esta sección antes de continuar.


In [1]:
# Instalar dependencias principales (versiones estables)
!pip -q install -U \
  "langchain==0.3.27" \
  "langchain-community==0.3.27" \
  "langchain-huggingface==0.3.1" \
  "transformers==4.55.2" \
  "huggingface_hub==0.34.4"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


In [2]:
from importlib.metadata import version, PackageNotFoundError

for dist in ["langchain", "langchain-community", "langchain-huggingface",
             "transformers", "huggingface_hub"]:
    try:
        print(dist, "=>", version(dist))
    except PackageNotFoundError:
        print(dist, "no instalado")

langchain => 0.3.27
langchain-community => 0.3.27
langchain-huggingface => 0.3.1
transformers => 4.55.2
huggingface_hub => 0.34.4


In [3]:
# Comprobar versión de Python y GPU/CPU
import sys, subprocess, torch
print(sys.version)
try:
    import torch
    print("PyTorch:", torch.__version__)
    print("CUDA disponible:", torch.cuda.is_available())
except Exception as e:
    print("PyTorch no disponible o sin CUDA", e)


3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
PyTorch: 2.8.0+cu126
CUDA disponible: False


### Configuración del token de Hugging Face

Para usar la Hugging Face Inference API, necesitás un token personal (gratuito). En Colab se recomienda guardarlo en `userdata`:

1. Crear el token en `https://huggingface.co/settings/tokens`.
2. En Colab: Abre el menu en la barra izquierda haciendo click en la llave → "Agregar nuevo secreto" ponle `HF_TOKEN` y el token que generamos → Abilita el acceso al notebook.
3. Ejecutar la celda siguiente para leerlo.


In [4]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')
assert HF_TOKEN is not None and len(HF_TOKEN) > 0, "Configurar el secreto 'HF_TOKEN' en Colab."
print("Token cargado OK: {HF_TOKEN}")


Token cargado OK: {HF_TOKEN}


## Parte 1 — Uso básico de LLMs con LangChain

Trabajaremos con un modelo instruct accesible vía Inference API. Para minimizar fricción, usaremos un modelo abierto y, cuando sea posible, una variante quantizada (el ya hosteado en HF).


In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

MODEL_ID = "Qwen/Qwen3-4B-Instruct-2507"

hf_endpoint = HuggingFaceEndpoint(
    repo_id=MODEL_ID,
    task="conversational",
    huggingfacehub_api_token="HF_TOKEN",
    temperature=0.7,
    max_new_tokens=256,
)

llm = ChatHuggingFace(llm=hf_endpoint)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente útil y conciso."),
    ("human", "Responde a la siguiente instrucción: {instruccion}"),
])

chain = prompt | llm | StrOutputParser()
print(chain.invoke({"instruccion": "Explica en 3 frases qué es un LLM y nombra 2 casos de uso."}))

Un LLM (Large Language Model) es un tipo de modelo de inteligencia artificial entrenado en grandes volúmenes de texto para entender, generar y realizar tareas relacionadas con el lenguaje humano.  
Casos de uso incluyen la redacción de contenido automatizada y asistencia en resolución de preguntas.  
Estos modelos también se utilizan en chatbots, traducción automática y generación de código.


### Ejercicio 1.1 (10 min)

- Probar 3 variaciones de `temperature` y observar el cambio en estilo.
- Cambiar el rol del `system` para forzar un estilo (p.ej., “responde con viñetas y máximo 3 líneas”).
- Pregunta sugerida: “Resume la diferencia entre entrenamiento y fine-tuning en 3 puntos.”


In [ ]:
# Ejercicio 1.1 — Variaciones de temperatura y estilo (compatible con nscale)
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

instruccion: str = "Resume la diferencia entre entrenamiento y fine-tuning en 3 puntos."

# Variar temperatura (crea un endpoint conversacional por cada T)
for t in [0.0, 0.7, 1.2]:
    tmp_endpoint = HuggingFaceEndpoint(
        repo_id=MODEL_ID,
        task="conversational",
        huggingfacehub_api_token="HF_TOKEN",
        temperature=t,
        max_new_tokens=256,
    )
    tmp_llm = ChatHuggingFace(llm=tmp_endpoint)
    respuesta = (prompt | tmp_llm | StrOutputParser()).invoke({"instruccion": instruccion})
    print("\n==== temperature:", t, "====\n", respuesta)

# Forzar estilo con system prompt (usa un LLM base conversacional)
base_endpoint = HuggingFaceEndpoint(
    repo_id=MODEL_ID,
    task="conversational",
    huggingfacehub_api_token=HF_TOKEN,
    temperature=0.7,
    max_new_tokens=256,
)
llm_base = ChatHuggingFace(llm=base_endpoint)

prompt_estilo = ChatPromptTemplate.from_messages([
    ("system", "Eres conciso. Responde SIEMPRE con viñetas y máximo 3 líneas."),
    ("human", "{instruccion}"),
])

respuesta_estilo = (prompt_estilo | llm_base | StrOutputParser()).invoke({"instruccion": instruccion})
print("\n==== estilo forzado ====:\n", respuesta_estilo)


==== temperature: 0.0 ====
 1. **Entrenamiento (training)**: Se refiere al proceso inicial de entrenar un modelo desde cero o a partir de un modelo preentrenado en grandes volúmenes de datos generales, para que aprenda características básicas de un dominio o tarea específica.

2. **Fine-tuning**: Consiste en ajustar un modelo ya entrenado (normalmente en un dominio general) con datos más específicos y relevantes para una tarea o dominio particular, para mejorar su rendimiento en esa tarea específica.

3. **Objetivo**: El entrenamiento busca aprender representaciones generales, mientras que el fine-tuning busca optimizar el modelo para una aplicación concreta, usando menos recursos y tiempo que un entrenamiento desde cero.

==== temperature: 0.7 ====
 1. **Entrenamiento (training)**: Consiste en entrenar un modelo desde cero o a partir de un modelo inicial, utilizando un conjunto de datos amplio y diverso para aprender patrones generales de una tarea o dominio.

2. **Fine-tuning**: Se 

## Parte 1.2 — Parámetros de decodificación

Ajustaremos parámetros como `top_p`, `repetition_penalty` y `max_new_tokens` para observar su efecto en la generación.


In [ ]:
# Exploración de parámetros de decodificación
from typing import Dict, Any
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser

consulta: str = "Escribe una analogía breve para explicar RAG a un público no técnico."

configuraciones: Dict[str, Dict[str, Any]] = {
    "baseline":   {"temperature": 0.7, "top_p": 0.95, "repetition_penalty": 1.0, "max_new_tokens": 128},
    "creativo":   {"temperature": 1.1, "top_p": 0.90, "repetition_penalty": 1.0, "max_new_tokens": 128},
    "controlado": {"temperature": 0.2, "top_p": 0.80, "repetition_penalty": 1.1, "max_new_tokens": 96},
}

for nombre, cfg in configuraciones.items():
    tmp_endpoint = HuggingFaceEndpoint(
        repo_id=MODEL_ID,
        task="conversational",
        huggingfacehub_api_token=HF_TOKEN,
        temperature=cfg["temperature"],
        top_p=cfg["top_p"],
        repetition_penalty=cfg["repetition_penalty"],
        max_new_tokens=cfg["max_new_tokens"],
    )
    tmp_llm = ChatHuggingFace(llm=tmp_endpoint)
    salida = (prompt | tmp_llm | StrOutputParser()).invoke({"instruccion": consulta})
    print(f"\n==== {nombre} ({cfg}) ====\n{salida}")


==== baseline ({'temperature': 0.7, 'top_p': 0.95, 'repetition_penalty': 1.0, 'max_new_tokens': 128}) ====
¡Claro! Aquí tienes una analogía breve y sencilla:

**RAG es como si tuvieras un libro enorme con información actualizada, y en lugar de memorizarlo todo, cada vez que necesitas una respuesta, buscas solo los párrafos más relevantes que te ayuden a responder con algo útil y preciso.**

Así, no se trata de recordar todo, sino de buscar la información necesaria en tiempo real.

==== creativo ({'temperature': 1.1, 'top_p': 0.9, 'repetition_penalty': 1.0, 'max_new_tokens': 128}) ====
Imagina que tienes un gran libro con millones de páginas, y necesitas responder una pregunta. En lugar de leer todo el libro, solo buscas las páginas más relevantes que tengan información sobre tu pregunta. El RAG (Retrieval-Augmented Generation) es como un sistema que busca rápidamente las páginas más útiles en un libro enorme (los datos) y luego las usa para crear una respuesta clara y precisa. Así, se

## Parte 2 — Prompt engineering avanzado

Exploraremos estrategias para mejorar la calidad y control de las respuestas: zero-shot, few-shot, restricciones de estilo, salida estructurada, Chain of Thought y Self-Consistency/Ensemble.


### Conceptos clave: Zero-shot, Few-shot, CoT, Roles y Self-Consistency

- Zero-shot: el modelo resuelve la tarea solo con instrucciones; no se proporcionan ejemplos.
- Few-shot: además de la instrucción, se incluyen 1-3 ejemplos que muestran el formato y estilo deseados.
- Chain-of-Thought (CoT): se guía al modelo para mostrar pasos intermedios de razonamiento (p.ej., “razona paso a paso”) antes de una respuesta final.
- Role prompting: se asigna un rol (p.ej., “actúa como profesor de IA”) para influir en el estilo y nivel de detalle.
- Self-Consistency / Ensemble: se generan múltiples respuestas estocásticas (con temperatura > 0) y se vota/selecciona la respuesta más consistente o frecuente.
- JSON output: se pide una salida estricta en formato JSON y se valida con un parser.

Lectura recomendada: guía de prompt engineering avanzada en `https://learnprompting.org/docs/introduction`.


### Zero-shot y Few-shot


In [ ]:
# Zero-shot vs Few-shot (diferencia marcada con patrón acróstico)
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LLM mas determinista
det_endpoint = HuggingFaceEndpoint(
    repo_id=MODEL_ID, task="conversational",
    huggingfacehub_api_token=HF_TOKEN, temperature=0.0, max_new_tokens=128
)
llm_det = ChatHuggingFace(llm=det_endpoint)
parser = StrOutputParser()

# Patrón no obvio: acróstico O-V-E (Overfitting), cada línea empieza con esa letra
instruccion = (
    "Escribe EXACTAMENTE 3 líneas sobre 'overfitting'. "
    "Cada línea debe comenzar con O, luego V, luego E (en ese orden). "
    "6-10 palabras por línea. Sin texto extra."
)

zero_shot = ChatPromptTemplate.from_messages([
    ("system", "Sigue estrictamente las instrucciones del usuario."),
    ("human", "{instruccion}")
])

few_shot = ChatPromptTemplate.from_messages([
    ("system", "Sigue estrictamente las instrucciones del usuario."),
    # Ejemplo: el patrón se demuestra con otro tema y otro acróstico (R-E-G)
    ("human", "Escribe EXACTAMENTE 3 líneas sobre 'regularización'. "
              "Cada línea debe comenzar con R, luego E, luego G. "
              "6-10 palabras por línea. Sin texto extra."),
    ("ai", "R Reduce complejidad para evitar ajustes al ruido\n"
           "E Estabiliza el aprendizaje con penalizaciones adecuadas\n"
           "G Generaliza mejor limitando pesos excesivamente grandes"),
    # Ahora se pide el caso real con el acróstico O-V-E
    ("human", "{instruccion}")
])

print("=== ZERO-SHOT ===")
print((zero_shot | llm_det | parser).invoke({"instruccion": instruccion}))

print("\n=== FEW-SHOT ===")
print((few_shot | llm_det | parser).invoke({"instruccion": instruccion}))

=== ZERO-SHOT ===
Overfitting occurs when a model learns noise instead of patterns.  
Overcomplicated models fail on new, unseen data.  
Overfitting reduces generalization in predictions.

=== FEW-SHOT ===
O Viola la generalización al ajustarse demasiado a datos de entrenamiento  
V Evalúa mal en datos nuevos por falta de robustez  
E Excede el poder de representación del modelo


### Role prompting

In [ ]:
# Role prompting
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

role_prompt = ChatPromptTemplate.from_messages([
    ("system", "Actúa como profesor de IA de nivel intermedio. Sé claro, estructurado y usa ejemplos sencillos."),
    ("human", "Explica brevemente qué es el aprendizaje por refuerzo y menciona 2 ejemplos de aplicación."),
])

print((role_prompt | llm | StrOutputParser()).invoke({}))


Claro, aquí tienes una explicación breve y clara:

**Qué es el aprendizaje por refuerzo (Reinforcement Learning):**  
Es un tipo de inteligencia artificial donde un agente (como un robot o un programa) aprende a tomar decisiones para maximizar una recompensa a lo largo del tiempo. No tiene acceso a datos etiquetados, sino que aprende probando diferentes acciones y viéndose recompensado o sancionado según los resultados.

🔍 **Por ejemplo:**  
Imagina que un agente quiere aprender a jugar al ajedrez. Cada vez que gana, recibe una recompensa. Si pierde, la recompensa es baja o negativa. Con el tiempo, el agente aprende qué movimientos le llevan a ganar más.

### 2 ejemplos de aplicación:
1. **Juegos de video** (como AlphaGo o Deep Q-Networks): Los algoritmos aprenden a jugar mejor jugando miles de partidas y recibiendo recompensas por ganar.
2. **Robótica y automatización** (como un robot que aprende a caminar o navegar): El robot recibe recompensas cuando se mueve bien o evita caídas, y 

### CoT (Chain-of-Thought) prompting

In [ ]:
# Prompts SIN CoT y CON CoT (Bayes) — nscale-compatible

from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

problema = (
    "En una población, 1% tiene la enfermedad. La prueba tiene 90% de sensibilidad y 90% de especificidad. "
    "Si una persona da positivo, ¿cuál es la probabilidad (en %) de que realmente esté enferma?"
)

parser = StrOutputParser()

# SIN CoT: SOLO porcentaje en una línea (recorta tokens)
endpoint_sin = HuggingFaceEndpoint(
    repo_id=MODEL_ID, task="conversational",
    huggingfacehub_api_token=HF_TOKEN, temperature=0.0, max_new_tokens=6
)
llm_sin = ChatHuggingFace(llm=endpoint_sin)

prompt_sin_cot = ChatPromptTemplate.from_messages([
    ("system", "Devuelve SOLO un número en formato porcentaje (ej: 8.33%). "
               "Sin explicaciones, sin ecuaciones, sin texto extra."),
    ("human", "{q}")
])

# CON CoT: piensa paso a paso y cierra con una línea final
endpoint_con = HuggingFaceEndpoint(
    repo_id=MODEL_ID, task="conversational",
    huggingfacehub_api_token=HF_TOKEN, temperature=0.0, max_new_tokens=256
)
llm_con = ChatHuggingFace(llm=endpoint_con)

prompt_con_cot = ChatPromptTemplate.from_messages([
    ("system", "Tómate tu tiempo y piensa paso a paso usando Bayes. "
               "Al final, da una sola línea con: 'Respuesta final: <n>%'"),
    ("human", "{q}")
])

print("=== SIN CoT ===")
print((prompt_sin_cot | llm_sin | parser).invoke({"q": problema}))

print("\n=== CON CoT ===")
print((prompt_con_cot | llm_con | parser).invoke({"q": problema}))

=== SIN CoT ===
9.09%

=== CON CoT ===
Vamos a resolver este problema paso a paso usando el **teorema de Bayes**.

---

### Datos del problema:

- Probabilidad de tener la enfermedad (prevalencia):  
  \( P(E) = 1\% = 0.01 \)

- Probabilidad de no tener la enfermedad:  
  \( P(\neg E) = 1 - 0.01 = 0.99 \)

- Sensibilidad de la prueba (probabilidad de dar positivo si tienes la enfermedad):  
  \( P(T^+ | E) = 90\% = 0.90 \)

- Especificidad de la prueba (probabilidad de dar negativo si no tienes la enfermedad):  
  \( P(T^- | \neg E) = 90\% = 0.90 \)

Esto implica que:

- El error de falsa positiva es \( P(T^+ | \neg E) = 1 - 0.90 = 0.10 \)

Nos piden:  
**Probabilidad de que una persona realmente tenga la enfermedad dado que dio positivo.**  
Es decir:  
\( P(E | T^+) = ? \)

---

### Usamos el teorema de Bayes:

\[
P(E | T^+) = \frac{P(T^+ | E) \cdot P(E)}{P(T^+)}
\]

Donde \( P(T^+) \) es la probabilidad total de dar positivo. Para calcularlo, usamos el **teorema de la probabilidad t

### Self-Consistency / Ensemble prompting

In [ ]:
# Ejemplo: self-consistency simple (compatible con nscale/conversational)
from collections import Counter
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser

pregunta = "Lista 3 riesgos comunes al usar LLMs en producción."
respuestas = []
parser = StrOutputParser()

for _ in range(5):
    tmp_endpoint = HuggingFaceEndpoint(
        repo_id=MODEL_ID,
        task="conversational",
        huggingfacehub_api_token=HF_TOKEN,
        temperature=1.0,
        top_p=0.9,
        max_new_tokens=200,
    )
    tmp_llm = ChatHuggingFace(llm=tmp_endpoint)
    texto = (prompt | tmp_llm | parser).invoke({"instruccion": pregunta})
    respuestas.append(texto.strip())

conteo = Counter(respuestas)
mejor_respuesta, _ = conteo.most_common(1)[0]

print("\n==== Candidatas ====")
for i, r in enumerate(respuestas, 1):
    print(f"[{i}]\n{r}\n")

print("==== Seleccionada por frecuencia ====")
print(mejor_respuesta)

KeyboardInterrupt: 

### Salida estructurada (JSON Output Parser)

In [ ]:
# Salida estructurada (JSON) con output parser
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

json_prompt = ChatPromptTemplate.from_template(
    """
    Eres un asistente que devuelve SIEMPRE JSON válido. Dado un tema, devuelve un objeto con:
    - "titulo": string
    - "puntos_clave": lista de 3 strings
    - "dificultad": uno de ["básico", "intermedio", "avanzado"]
    Responde SOLO con JSON válido sin texto adicional.
    Tema: {tema}
    """
)

json_text = (json_prompt | llm | StrOutputParser()).invoke({"tema": "RAG"})
print(json_text)

data = json.loads(json_text)
assert set(["titulo", "puntos_clave", "dificultad"]).issubset(data.keys())
print("JSON válido con las claves requeridas.")


{
  "titulo": "RAG (Retrieval-Augmented Generation)",
  "puntos_clave": [
    "RAG combina la recuperación de información de una base de conocimientos con la generación de texto para mejorar la precisión y relevancia.",
    "Permite que los modelos de lenguaje utilicen información de documentos externos en tiempo real durante la generación de respuestas.",
    "Reduce el riesgo de generar contenido fabricado al basarse en fuentes verificadas de información."
  ],
  "dificultad": "intermedio"
}
JSON válido con las claves requeridas.


## Ejercicios — Parte 2

Resuelve los siguientes ejercicios. Modifica prompts y parámetros si es necesario y justifica brevemente tus decisiones (en una celda de texto).


### Ejercicio 2.1 — Zero-shot vs Few-shot

- Tarea: explicar “regularización L2” en 3 viñetas claras para un público técnico.
- Paso 1 (zero-shot): crea un prompt sin ejemplos y observa el resultado.
- Paso 2 (few-shot): agrega 1-2 ejemplos de estilo y compara la salida.
- Pregunta guía: ¿mejoró la precisión o claridad con pocos ejemplos? Justifica.


In [ ]:
# Ejercicio 2.1 — Zero-shot vs Few-shot (código)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Zero-shot
zero_shot_A = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente técnico y claro. Responde en viñetas."),
    ("human", "Explica 'regularización L2' en 3 viñetas para un público técnico."),
])
print("\n=== ZERO-SHOT ===\n")
print((zero_shot_A | llm | StrOutputParser()).invoke({}))

# Few-shot (agregar 1-2 ejemplos de estilo)
few_shot_A = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente técnico y claro. Responde en viñetas."),
    ("human", "Explica 'regularización L1' en 3 viñetas"),
    ("ai", "- Penaliza magnitud absoluta\n- Induce sparsidad en coeficientes\n- Útil para selección de variables"),
    ("human", "Explica 'regularización L2' en 3 viñetas"),
])
print("\n=== FEW-SHOT ===\n")
print((few_shot_A | llm | StrOutputParser()).invoke({}))


=== ZERO-SHOT ===

- La **regularización L2** añade un término al costo de pérdida (loss function) que penaliza el tamaño de los coeficientes del modelo, específicamente sumando el cuadrado de todos los pesos, multiplicado por un parámetro de regularización λ.  
- Este enfoque tiende a **reducir el overfitting** al promover soluciones con coeficientes más pequeños y distribuidos, lo que implica una modelación más generalizable.  
- En términos matemáticos, si la función de pérdida original es $ \mathcal{L} $, la versión regularizada se expresa como $ \mathcal{L} + \lambda \sum_{i} w_i^2 $, donde $ w_i $ son los coeficientes del modelo y $ \lambda $ controla el grado de penalización.

=== FEW-SHOT ===

- Penaliza cuadrado de magnitud de coeficientes  
- Promueve coeficientes pequeños pero no nulos  
- Ayuda a evitar sobreajuste en modelos de regresión


### Ejercicio 2.2 — Chain-of-Thought (CoT)

- Tarea: dado un problema de evaluación de modelos, razonar paso a paso y entregar una conclusión final breve.
- Problema sugerido: “¿Por qué accuracy puede ser engañoso en un dataset muy desbalanceado y qué métrica alternativa usarías?”
- Pista: pide explícitamente “razona paso a paso y luego da una respuesta final breve en una línea”.


In [ ]:
# Ejercicio 2.2 — CoT (código)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

cot_ex = ChatPromptTemplate.from_messages([
    ("system", "Eres un tutor de IA. Razona paso a paso y luego da una respuesta final breve."),
    ("human", "Problema: {problema}\nIndica tus pasos de razonamiento y luego una respuesta final en una sola línea."),
])

problema = "¿Por qué accuracy puede ser engañoso en un dataset muy desbalanceado y qué métrica alternativa usarías?"
print((cot_ex | llm | StrOutputParser()).invoke({"problema": problema}))


Paso 1: En un dataset desbalanceado, la mayoría de las instancias pertenecen a una clase, por lo que un modelo puede predecir solo esa clase con alta frecuencia y obtener una accuracy alta, aunque sea inútil prácticamente.

Paso 2: Esta alta accuracy es engañosa porque no refleja el desempeño real en las clases minoritarias, que suelen ser más importantes en aplicaciones reales.

Paso 3: Una métrica alternativa que considera mejor el desempeño en todas las clases es la precisión (precision), el recall (o tasa de detección), o el F1-score, especialmente el F1-score, que es el promedio armonicamente entre precisión y recall.

Paso 4: El F1-score ofrece un balance adecuado entre precisión y recall, y es más robusto en datos desbalanceados.

Respuesta final: La accuracy puede ser engañoso en datasets desbalanceados porque favorece la clase mayoritaria; se recomienda usar el F1-score como métrica alternativa.


### Ejercicio 2.3 — Role prompting

- Tarea: explicar el “sesgo de selección” a un equipo de data engineering con ejemplos concisos.
- Rol: “Actúa como líder técnico de datos; sé pragmático y directo, con viñetas concretas”.
- Objetivo: evaluar cómo cambia el estilo bajo un rol técnico específico.


In [ ]:
# Ejercicio 2.3 — Role prompting (código)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

role_ex = ChatPromptTemplate.from_messages([
    ("system", "Actúa como líder técnico de datos; sé pragmático y directo, con viñetas concretas."),
    ("human", "Explica el ‘sesgo de selección’ a un equipo de data engineering en 3 viñetas y da 1 ejemplo práctico."),
])

print((role_ex | llm | StrOutputParser()).invoke({}))


- **El sesgo de selección ocurre cuando los datos usados para entrenar un modelo o tomar decisiones no representan adecuadamente a toda la población objetivo**, lo que lleva a predicciones o conclusiones sesgadas.  
- **En data engineering, esto suele manifestarse en la forma en que se recolectan, filtran o procesan los datos**: por ejemplo, solo usar registros de usuarios activos en una aplicación, ignorando a los inactivos.  
- **El riesgo es que los modelos se entrenen en subconjuntos distorsionados**, resultando en decisiones que no se generalizan bien (como un algoritmo de recomendación que nunca sugiere contenidos para usuarios pasivos).

**Ejemplo práctico**:  
Un equipo de datos construye un sistema de detección de fraudes que solo se entrena con transacciones de usuarios de cierto país (por ejemplo, EE.UU.). Como el perfil de transacciones y comportamientos de fraudes varían en otros países, el modelo falla al detectar fraudes en transacciones de usuarios de España, por lo que